In [26]:
import csv
import numpy as np
import torch
#Load the data
bikes_numpy = np.loadtxt("/Users/rachagouareb/Desktop/HES-SO/MaterialPreStarting/Bike-Sharing-Dataset/hour.csv",
dtype=np.float32,
delimiter=",",
skiprows=1,
converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
bikes,bikes.size()

(tensor([[1.0000e+00, 1.1000e+01, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
          1.6000e+01],
         [2.0000e+00, 1.1000e+01, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
          4.0000e+01],
         [3.0000e+00, 1.1000e+01, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
          3.2000e+01],
         ...,
         [1.7374e+04, 1.2000e+01, 1.0000e+00,  ..., 1.0000e+01, 1.1200e+02,
          1.2200e+02],
         [1.7375e+04, 1.2000e+01, 1.0000e+00,  ..., 1.1000e+01, 1.0800e+02,
          1.1900e+02],
         [1.7376e+04, 1.2000e+01, 1.0000e+00,  ..., 8.0000e+00, 8.1000e+01,
          8.9000e+01]]), torch.Size([17376, 17]))

In [29]:
#view the same tensor in batches of 24 hours
#View method here allow you to rearrange your tensor at zero cost because no data is copied at all
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([724, 24, 17]), (408, 17, 1))

In [30]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([724, 17, 24]), (408, 1, 17))

In [38]:
#weather-situation variable is ordinal
#limit yourself to the first day for now
#initialize a zero-filled matrix with a number of rows equal to the number of hours in
#the day and a number of columns equal to the number of weather levels
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
#9 for the weather column
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [39]:
#scatter ones into our matrix according to the corresponding level at each row
#Use of unsqueeze to add a singleton dimension earlier
#Decreasing the values by 1 because the weather situation ranges from 1 to 4, whereas indices are 0-based
the weather situation ranges from 1 to 4,
whereas indices are 0-based
weather_onehot.scatter_(
dim=1,
index=first_day[:,9].unsqueeze(1) - 1,
value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [53]:
#Concatenate your matrix to your original data set, using the cat function
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000, 11.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [54]:
#You could have done the same thing with the reshaped daily_bikes tensor
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([724, 4, 24])

In [55]:
#Then scatter the one-hot encoding into the tensor in the C dimension
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1,
1.0)
daily_weather_onehot.shape

torch.Size([724, 4, 24])

In [57]:
#Concatenate along the C dimension
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)